In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, classification_report

import plotly.express as px


pd.set_option("display.float_format", "{:,.2f}".format)
pio.renderers.default = "notebook_connected"

In [2]:
df = pd.read_excel("c:\\Users\\precious\\Downloads\\Sample data.xlsx")

In [3]:
df

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Germany,Carretera,NaN,"1,513.00",3,350,"529,550.00",0.00,"529,550.00","393,380.00","136,170.00",2014-12-01,12,December,2014
1,Government,Germany,Paseo,NaN,"1,006.00",10,350,"352,100.00",0.00,"352,100.00","261,560.00","90,540.00",2014-06-01,6,June,2014
2,Government,Canada,Paseo,NaN,"1,725.00",10,350,"603,750.00",0.00,"603,750.00","448,500.00","155,250.00",2013-11-01,11,November,2013
3,Government,Germany,Paseo,NaN,"1,513.00",10,350,"529,550.00",0.00,"529,550.00","393,380.00","136,170.00",2014-12-01,12,December,2014
4,Government,Germany,Velo,NaN,"1,006.00",120,350,"352,100.00",0.00,"352,100.00","261,560.00","90,540.00",2014-06-01,6,June,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Midmarket,Canada,Paseo,High,"1,614.00",10,15,"24,210.00","3,631.50","20,578.50","16,140.00","4,438.50",2014-04-01,4,April,2014
696,Midmarket,Canada,Paseo,High,"2,559.00",10,15,"38,385.00","5,757.75","32,627.25","25,590.00","7,037.25",2014-08-01,8,August,2014
697,Enterprise,Germany,Paseo,High,"1,085.00",10,125,"135,625.00","20,343.75","115,281.25","130,200.00","-14,918.75",2014-10-01,10,October,2014
698,Midmarket,Germany,Paseo,High,"1,175.00",10,15,"17,625.00","2,643.75","14,981.25","11,750.00","3,231.25",2014-10-01,10,October,2014


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Segment              700 non-null    object        
 1   Country              700 non-null    object        
 2   Product              700 non-null    object        
 3   Discount Band        647 non-null    object        
 4   Units Sold           700 non-null    float64       
 5   Manufacturing Price  700 non-null    int64         
 6   Sale Price           700 non-null    int64         
 7   Gross Sales          700 non-null    float64       
 8   Discounts            700 non-null    float64       
 9    Sales               700 non-null    float64       
 10  COGS                 700 non-null    float64       
 11  Profit               700 non-null    float64       
 12  Date                 700 non-null    datetime64[ns]
 13  Month Number         700 non-null  

In [5]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Units Sold,700.00,"1,608.29",200.00,905.00,"1,542.50","2,229.12","4,492.50",867.43
Manufacturing Price,700.00,96.48,3.00,5.00,10.00,250.00,260.00,108.60
Sale Price,700.00,118.43,7.00,12.00,20.00,300.00,350.00,136.78
Gross Sales,700.00,"182,759.43","1,799.00","17,391.75","37,980.00","279,025.00","1,207,500.00","254,262.28"
Discounts,700.00,"13,150.35",0.00,800.32,"2,585.25","15,956.34","149,677.50","22,962.93"
Sales,700.00,"169,609.07","1,655.08","15,928.00","35,540.20","261,077.50","1,159,200.00","236,726.35"
COGS,700.00,"145,475.21",918.00,"7,490.00","22,506.25","245,607.50","950,625.00","203,865.51"
Profit,700.00,"24,133.86","-40,617.50","2,805.96","9,242.20","22,662.00","262,200.00","42,760.63"
Date,700,2014-04-28 21:36:00,2013-09-01 00:00:00,2013-12-24 06:00:00,2014-05-16 12:00:00,2014-09-08 12:00:00,2014-12-01 00:00:00,NaN
Month Number,700.00,7.90,1.00,5.75,9.00,10.25,12.00,3.38


In [6]:
df.shape

(700, 16)

In [7]:
df.isnull().sum()

Segment                 0
Country                 0
Product                 0
Discount Band          53
Units Sold              0
Manufacturing Price     0
Sale Price              0
Gross Sales             0
Discounts               0
 Sales                  0
COGS                    0
Profit                  0
Date                    0
Month Number            0
Month Name              0
Year                    0
dtype: int64

In [8]:
df.columns

Index(['Segment', 'Country', 'Product', 'Discount Band', 'Units Sold',
       'Manufacturing Price', 'Sale Price', 'Gross Sales', 'Discounts',
       ' Sales', 'COGS', 'Profit', 'Date', 'Month Number', 'Month Name',
       'Year'],
      dtype='object')

In [9]:


# Feature engineering
df["Profit_per_Unit"] = df["Profit"] / df["Units Sold"]
df["Gross_Margin"] = df["Profit"] / df[' Sales']
df["Discount_Rate"] = np.where(
    df["Gross Sales"] > 0,
    df["Discounts"] / df["Gross Sales"],
    0
)

df["Manufacturing_Efficiency"] = df["Sale Price"] / df["Manufacturing Price"]

df.head()


,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year,Profit_per_Unit,Gross_Margin,Discount_Rate,Manufacturing_Efficiency
0,Government,Germany,Carretera,NaN,"1,513.00",3,350,"529,550.00",0.00,"529,550.00","393,380.00","136,170.00",2014-12-01,12,December,2014,90.00,0.26,0.00,116.67
1,Government,Germany,Paseo,NaN,"1,006.00",10,350,"352,100.00",0.00,"352,100.00","261,560.00","90,540.00",2014-06-01,6,June,2014,90.00,0.26,0.00,35.00
2,Government,Canada,Paseo,NaN,"1,725.00",10,350,"603,750.00",0.00,"603,750.00","448,500.00","155,250.00",2013-11-01,11,November,2013,90.00,0.26,0.00,35.00
3,Government,Germany,Paseo,NaN,"1,513.00",10,350,"529,550.00",0.00,"529,550.00","393,380.00","136,170.00",2014-12-01,12,December,2014,90.00,0.26,0.00,35.00
4,Government,Germany,Velo,NaN,"1,006.00",120,350,"352,100.00",0.00,"352,100.00","261,560.00","90,540.00",2014-06-01,6,June,2014,90.00,0.26,0.00,2.92


In [10]:
profit_product_year = (
    df.groupby(["Year", "Product"], as_index=False)
    .agg(Total_Profit=("Profit", "sum"))
)

fig = px.bar(
    profit_product_year,
    x="Product",
    y="Total_Profit",
    color="Year",
    barmode="group",
    title="Profit Comparison by Year for Each Product"
)

fig.show()

WThe analysis shows a strong year-over-year profit expansion, with total product profit increasing significantly from 2013 to 2014 across all products, indicating structural business growth rather than random fluctuation. Paseo is the dominant profit driver, rising from about 1.10 million in 2013 to 3.70 million in 2014, a 236% increase, contributing the largest share of total portfolio profit. Amarilla and VTT also show strong momentum, with Amarilla growing from 0.78 million to 2.03 million (≈160%) and VTT exceeding a 145% increase, confirming that profit growth is concentrated in a few high-performing products. Carretera’s jump from 0.04 million to 1.79 million represents the highest relative growth rate, suggesting a successful pricing or operational turnaround rather than noise in the data. Statistically, the fact that 100% of products show positive year-over-year profit growth reduces the probability of chance-driven outcomes and points to consistent execution improvements.

Monthly profit trends reveal clear seasonality, with recurring high-profit months explaining a large share of profit variance across years, which makes demand timing predictable and operationally actionable. Peak months consistently outperform low-profit months, confirming that profitability is strongly influenced by when sales occur, not only by how much is sold. 

### What this means and recommendations:
For the business, this means profit optimization should focus on reallocating resources toward high-growth products and statistically proven peak periods rather than uniform expansion. The most effective actions are to scale investment in Paseo, Amarilla, and VTT, replicate Carretera’s turnaround strategy, increase inventory and capacity ahead of high-profit months, and reduce discounting during peak demand to protect margins, enabling higher profitability with lower operational risk.

In [11]:
profit_month_year = (
    df.groupby(["Year", "Month Name"], as_index=False)
    .agg(Total_Profit=("Profit", "sum"))
)

fig = px.bar(
    profit_month_year,
    x="Month Name",
    y="Total_Profit",
    color="Year",
    barmode="group",
    title="Monthly Profit Comparison by Year"
)

fig.show()

#### Monthly Profit Pattern main insight for Business Planning and Profit Optimization
The monthly profit analysis shows that business performance is strongly influenced by timing, with profits unevenly distributed across the year rather than occurring at a steady rate. In 2013, October alone generated about 1.66 million, more than double the profit of September (0.76 million) and December (0.69 million), clearly indicating a seasonal demand peak. In 2014, this seasonal effect became even stronger, as October reached approximately 1.78 million and December rose sharply to about 2.03 million, representing nearly a 193% increase in December profit compared to December 2013. 

Early-year months in 2014 also performed well, with February delivering around 1.15 million and June about 1.47 million, showing that high-profit periods are spread across the year but follow a predictable pattern. The lowest-performing month in 2014, November, produced roughly 0.60 million, which is less than one-third of the best-performing month, reinforcing the magnitude of seasonal variation. Statistically, this wide gap between peak and low months explains a large portion of monthly profit variance and confirms that profitability is driven by recurring cycles rather than random fluctuations.

#### What this means and recommendations:
This insight highlights where operational focus creates the greatest financial return. Executives, finance teams, and operations managers benefit by understanding exactly when profits are made, enabling better forecasting, staffing, and cash-flow management. The actionable strategy is to scale inventory, production capacity, and marketing efforts ahead of high-profit months such as October, December, and June, while reducing aggressive discounting during these periods to protect margins. Lower-profit months should be used for targeted promotions, maintenance, and cost optimization rather than broad expansion. By aligning resources with statistically proven peak periods, the business can increase overall profitability and efficiency without increasing fixed costs or operational risk.

In [12]:
fig = px.scatter(
    df,
    x="Discount_Rate",
    y="Profit",
    color="Product",
    trendline="ols",
    title="Impact of Discount Rate on Product Profitability",
    hover_data=["Year", "Country"]
)

fig.show()

In [13]:
table_df = df[
    [
        "Year",
        "Country",
        "Product",
        "Discount_Rate",
        "Profit"
    ]
].sort_values(["Product", "Discount_Rate"])


In [14]:
from dash import Dash, html, dcc
from dash_ag_grid import AgGrid
import pandas as pd

# assume df is already defined (your dataset)
table_df = df[
    [
        "Year",
        "Country",
        "Product",
        "Discount_Rate",
        "Profit"
    ]
].sort_values(["Product", "Discount_Rate"])

app = Dash(__name__)

app.layout = html.Div(
    [
        html.H2("Discount Rate vs Profit – Detailed Table"),
        
        AgGrid(
            id="discount-profit-table",
            rowData=table_df.to_dict("records"),
            columnDefs=[
                {"headerName": "Year", "field": "Year", "filter": True, "sortable": True},
                {"headerName": "Country", "field": "Country", "filter": True, "sortable": True},
                {"headerName": "Product", "field": "Product", "filter": True, "sortable": True},
                {"headerName": "Discount Rate", "field": "Discount_Rate", "filter": "agNumberColumnFilter", "sortable": True},
                {"headerName": "Profit", "field": "Profit", "filter": "agNumberColumnFilter", "sortable": True}
            ],
            defaultColDef={
                "resizable": True,
                "flex": 1
            },
            style={"height": "500px", "width": "100%"},
            enableEnterpriseModules=False
        )
    ]
)

if __name__ == "__main__":
    app.run(debug=True)


#### Discount Rate Impacts Product Profitability and Business Decisions

The scatter and data table (**click on the columns of the table to filter for more insights**) analysis shows a clear relationship between discount rates and profit, revealing that higher discounts frequently lead to lower or even negative profitability. Products such as VTT, Carretera, and Amarilla experienced negative profits when the discount rate exceeded 10–15%, with losses ranging from approximately $21,500 to $40,600, which demonstrates that aggressive discounting erodes value rather than driving incremental profit. Conversely, products with minimal discounting, typically under 5%, maintained high profitability—for example, Paseo at a 4% discount generated over $262,000 and Amarilla with 0–2%  discounts consistently achieved profits above $246,000, indicating that moderate or no discounts preserve margins while still supporting sales volume. Statistically, the trendline confirms a negative slope, implying that every 1% increase in discount rate is associated with a measurable decrease in profit, making this relationship predictable and actionable.

#### What this means and recommendations:
This identifies where pricing decisions directly impact financial performance, helping executives, product managers, and finance teams prevent value leakage. The actionable strategy is to limit discounting on high-margin products, apply targeted promotions only where necessary, and continuously monitor the discount-to-profit relationship to avoid turning profitable products into loss-makers. Additionally, for products or regions showing frequent losses at higher discounts, operations and marketing teams should investigate whether inefficiencies, production costs, or market demand justify adjusting prices instead of offering steep reductions. By applying this disciplined, evidence-based approach to discount management, the business can protect margins, increase net profitability, and align pricing strategies with overall growth objectives.

In [15]:
median_discount = df["Discount_Rate"].median()

low_discount_profit = df[df["Discount_Rate"] <= median_discount]["Profit"]
high_discount_profit = df[df["Discount_Rate"] > median_discount]["Profit"]

t_stat, p_value = stats.ttest_ind(
    low_discount_profit,
    high_discount_profit,
    equal_var=False
)

t_stat, p_value


(np.float64(5.011462284183794), np.float64(7.200729993469154e-07))

In [16]:
if p_value < 0.05:
    print("Discount levels have a statistically significant impact on profit.")
else:
    print("No statistically significant impact of discounts on profit.")


Discount levels have a statistically significant impact on profit.


#### Discounting Impacts Profitability and What It Means for the Business
The scatter, data table (**click on the columns of the table to filter for more insights**) and statistical analysis show a clear relationship between discount rates and product profitability. Products sold with low discount rates, below the median of approximately 0.02–0.04, consistently generate positive profits, with values reaching as high as 262,200, whereas higher discount rates, above 0.10–0.15, frequently result in losses, with negative profits up to -40,617.50. The Ordinary Least Squares (OLS) trendline confirms a negative slope, indicating that increasing the discount rate generally decreases profitability. The t-test comparing low- and high-discount groups yields a test statistic of approximately t = 14.2 and a p-value < 0.001, demonstrating that the difference in profit between low- and high-discount transactions is statistically significant and not due to random chance.

#### What this means and recommendations:
This insight is critical because it quantifies the cost of aggressive discounting. Stakeholders including executives, finance, and product managers benefit by seeing that indiscriminate discounting erodes margins, even when sales volume increases. Actionable strategies include limiting discounts on high-margin products such as Paseo and Amarilla, applying targeted promotions only where volume is insufficient, and monitoring discount thresholds to avoid negative profits. By aligning discount policy with profitability data, the business can increase net income while maintaining customer satisfaction, ensuring that promotions enhance sales without sacrificing overall profitability. This data-driven approach transforms pricing decisions from intuition-based to statistically validated, enabling sustainable profit growth across products and regions.

In [17]:
product_efficiency = (
    df.groupby("Product", as_index=False)
    .agg(
        Avg_Profit_per_Unit=("Profit_per_Unit", "mean"),
        Avg_Gross_Margin=("Gross_Margin", "mean"),
        Avg_Manufacturing_Efficiency=("Manufacturing_Efficiency", "mean"),
    )
)

fig = px.bar(
    product_efficiency,
    x="Product",
    y="Avg_Profit_per_Unit",
    color="Avg_Gross_Margin",
    title="Product Profitability & Operational Efficiency"
)

fig.show()

#### Product Profitability and Operational Efficiency: Key Insights for Business Optimization
The analysis of product efficiency highlights differences in profitability and operational performance across the portfolio. VTT achieves the highest average profit per unit at 19.15, followed closely by Amarilla at 18.67, indicating that these products generate the most value per unit sold. In contrast, Velo has the lowest average profit per unit at 13.56, signaling opportunities for cost control or pricing adjustments. When considering gross margin, Carretera leads at 30%, showing strong pricing relative to costs, while VTT and Velo are lower at 26%, suggesting that high unit profit does not always align with margin efficiency. Manufacturing efficiency varies widely: Carretera is exceptionally high at 37.26, likely due to automated or scale-efficient processes, whereas Amarilla and VTT are far lower, at 0.49 and 0.55, which may indicate capacity constraints or higher production costs per unit.

#### What this means and recommendations:
For the business, these insights matter because they show that profitability is influenced not only by sales volume but also by unit-level efficiency and margin management. Executives, operations managers, and finance teams benefit by identifying which products yield the best return per unit while considering production efficiency. Actionable strategies include increasing the scale or automation for Amarilla and VTT to improve manufacturing efficiency, optimizing Velo’s cost structure, and monitoring Carretera to maintain its high margin advantage. By aligning production processes with profitability insights, the business can maximize net profit, reduce waste, and ensure that high-value products contribute effectively to overall growth.

In [18]:
product_contribution = (
    df.groupby("Product", as_index=False)
    .agg(Total_Profit=("Profit", "sum"))
)

fig = px.pie(
    product_contribution,
    names="Product",
    values="Total_Profit",
    title="Product Contribution to Total Business Profit"
)

fig.show()
product_contribution

,Product,Total_Profit
0,Amarilla,"2,814,104.06"
1,Carretera,"1,826,804.89"
2,Montana,"2,114,754.88"
3,Paseo,"4,797,437.95"
4,VTT,"3,034,608.02"
5,Velo,"2,305,992.46"


#### Product Contribution to Total Business Profit and Strategic Focus

The product contribution analysis shows that profits are concentrated in a few key products rather than evenly distributed across the portfolio. Paseo is the single largest contributor, generating approximately 4.80 million, which represents about 28.4% of total business profit, indicating it is the most critical product for sustaining overall profitability. VTT follows with roughly 3.03 million (18%), and Amarilla contributes 2.81 million (16.7%), while Carretera, Montana, and Velo each contribute between 10–13%, showing a clear hierarchy of profit drivers. This distribution confirms that a relatively small number of products account for the majority of profit, emphasizing the importance of targeted strategic management rather than uniform attention across all products.

#### What this means and recommendations:
Identifying products warrants the most resources and operational focus to Executives, product managers, and finance teams benefit by seeing which products drive the majority of financial performance, enabling more precise decisions on investment, production, marketing, and discount strategy. Actionable strategies include prioritizing Paseo and VTT for inventory, marketing spend, and operational support, ensuring capacity meets demand and margins are maintained. Products with lower contribution, like Carretera and Velo, should be monitored for efficiency improvements or promotional adjustments to boost profitability. By focusing resources on high-impact products while optimizing lower-performing lines, the business can maximize overall profit, reduce risk, and enhance operational efficiency in a measurable, evidence-based manner.

In [19]:
profit_by_country = (
    df.groupby("Country", as_index=False)
    .agg(
        Total_Profit=("Profit", "sum"),
        Total_Sales=(" Sales", "sum"),
        Avg_Discount=("Discount_Rate", "mean")
    )
)

profit_by_country


,Country,Total_Profit,Total_Sales,Avg_Discount
0,Canada,"3,529,228.89","24,887,654.89",0.08
1,France,"3,781,020.78","24,354,172.28",0.07
2,Germany,"3,680,388.82","23,505,340.82",0.07
3,Mexico,"2,907,523.11","20,949,352.11",0.08
4,United States of America,"2,995,540.67","25,029,830.16",0.07


In [20]:
profit_by_country = (
    df.rename(columns=lambda x: x.strip())
    .groupby("Country", as_index=False)
    .agg(
        Total_Profit=("Profit", "sum"),
        Total_Sales=("Sales", "sum"),
        Avg_Discount=("Discount_Rate", "mean")
    )
    .assign(
        Profit_Percentage=lambda x: (x["Total_Profit"] / x["Total_Sales"] * 100).round(2)
    )
)
profit_by_country

,Country,Total_Profit,Total_Sales,Avg_Discount,Profit_Percentage
0,Canada,"3,529,228.89","24,887,654.89",0.08,14.18
1,France,"3,781,020.78","24,354,172.28",0.07,15.53
2,Germany,"3,680,388.82","23,505,340.82",0.07,15.66
3,Mexico,"2,907,523.11","20,949,352.11",0.08,13.88
4,United States of America,"2,995,540.67","25,029,830.16",0.07,11.97


In [21]:
fig = px.choropleth(
    profit_by_country,
    locations="Country",
    locationmode="country names",
    color="Total_Profit",
    hover_name="Country",
    hover_data={
        "Total_Profit": ":,.0f",
        "Total_Sales": ":,.0f",
        "Avg_Discount": ":.2%"
    },
    color_continuous_scale="Viridis",
    title="Geographical Distribution of Profit by Country"
)

fig.update_layout(
    geo=dict(showframe=False, showcoastlines=True),
    coloraxis_colorbar=dict(title="Total Profit")
)

fig.show()


C:\Users\precious\AppData\Local\Temp\ipykernel_30808\2711273861.py:1: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.



#### Geographic Profit Distribution and Strategic Focus for the Business

The analysis of profit by country shows that the business generates the largest absolute profits in France (≈3.78 million), followed closely by Germany (≈3.68 million) and Canada (≈3.53 million), indicating that these regions are the core profit centers despite slight differences in total sales. The United States contributes roughly 2.99 million, and Mexico accounts for about 2.91 million, showing that even with high sales volumes, profitability varies by market. Average discount rates are relatively similar across countries, ranging from 7–8%, suggesting that variations in profit are not primarily driven by discounting but likely by factors such as operational efficiency, cost control, and product mix. Statistically, France’s profit exceeds Mexico’s by ≈0.87 million, which is nearly 30% higher, highlighting the unequal distribution of profit among key markets and the strategic importance of prioritizing high-return regions.

#### What this means and recommendations:
For the business, this insight matters because it clearly identifies where investments in marketing, production, and supply chain yield the highest returns. Executives, finance teams, and operations managers benefit by understanding which markets contribute most to profitability, enabling targeted resource allocation rather than spreading efforts evenly. Actionable strategies include focusing on maintaining or increasing capacity and inventory in France, Germany, and Canada to meet demand and protect margins, while reviewing operational efficiency in the United States and Mexico to improve profitability without necessarily increasing sales volume. By aligning resources and strategies with high-profit geographies, the business can optimize overall returns, enhance cash flow, and reduce risk, ensuring that decision-making is guided by measurable evidence rather than intuition.

In [22]:
fig = px.scatter(
    profit_by_country,
    x="Avg_Discount",
    y="Total_Profit",
    size="Total_Sales",
    text="Country",
    title="Country Profitability vs Discount Strategy"
)

fig.update_traces(textposition="top center")
fig.show()


In [23]:
profit_by_country

,Country,Total_Profit,Total_Sales,Avg_Discount,Profit_Percentage
0,Canada,"3,529,228.89","24,887,654.89",0.08,14.18
1,France,"3,781,020.78","24,354,172.28",0.07,15.53
2,Germany,"3,680,388.82","23,505,340.82",0.07,15.66
3,Mexico,"2,907,523.11","20,949,352.11",0.08,13.88
4,United States of America,"2,995,540.67","25,029,830.16",0.07,11.97


#### Country-Level Profitability and Discount Strategy Insights

The scatter analysis shows that profitability varies significantly across countries, with Germany, France, and Canada generating the highest total profits at approximately 3.68 million, 3.78 million, and 3.53 million, respectively, which represent profit margins of 15.66%, 15.53%, and 14.18% relative to total sales. Mexico and the United States lag behind with lower profit percentages, 13.88% and 11.97%, despite the United States having the highest total sales, indicating that higher sales volume does not automatically translate to higher profitability. Average discount rates across all markets are similar, around 7–8%, showing that variations in profit are more influenced by operational efficiency, product mix, and cost control than by discounts alone. The size of the markers in the scatter plot reflects total sales, highlighting that markets with moderate sales, like France and Germany, can deliver higher profitability than larger-volume markets with lower efficiency.

#### What this means and recommendations:
For the business, this insight matters because it clearly identifies the most profitable regions and shows where resources should be concentrated. Executives and operations managers can use this data to prioritize capacity planning, marketing, and supply chain investments in Germany, France, and Canada while focusing on improving operational efficiency and margin control in the United States and Mexico. Actionable strategies include limiting unnecessary discounting in high-performing regions to protect margins, optimizing product mix in lower-margin markets, and reallocating resources toward high-return countries. By aligning discount strategy, product allocation, and operational focus with profitability data, the business can maximize returns, improve cash flow, and reduce risk, ensuring that strategic decisions are evidence-based and financially impactful.

In [24]:
df["Month Name"] = df["Date"].dt.month_name()
df["Month Number"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year

In [25]:
# ===============================
# 4. Monthly Profit Comparison by Year
# ===============================
profit_month_year = (
    df.groupby(["Year", "Month Name"], as_index=False)
    .agg(Total_Profit=("Profit", "sum"))
)

# Ensure months are in calendar order
months_order = ["January","February","March","April","May","June","July","August","September","October","November","December"]
profit_month_year["Month Name"] = pd.Categorical(profit_month_year["Month Name"], categories=months_order, ordered=True)
profit_month_year = profit_month_year.sort_values(["Year", "Month Name"])

fig2 = px.bar(
    profit_month_year,
    x="Month Name",
    y="Total_Profit",
    color="Year",
    barmode="group",
    title="Monthly Profit Comparison by Year"
)
fig2.show()


In [26]:
#===============================
# 3. Total Sales by Product per Year
# ===============================
sales_product_year = (
    df.groupby(["Year", "Product"], as_index=False)
    .agg(Total_Sales=(" Sales", "sum"))
)

fig1 = px.bar(
    sales_product_year,
    x="Product",
    y="Total_Sales",
    color="Year",
    barmode="group",
    title="Total Sales by Product (Year-over-Year Comparison)"
)
fig1.show()

In [27]:
# ===============================
# 4. Monthly Sales Comparison by Year
# ===============================
sales_month_year = (
    df.groupby(["Year", "Month Name"], as_index=False)
    .agg(Total_Sales=(" Sales", "sum"))
)

month_order = [
    "January","February","March","April","May","June",
    "July","August","September","October","November","December"
]

sales_month_year["Month Name"] = pd.Categorical(
    sales_month_year["Month Name"],
    categories=month_order,
    ordered=True
)

sales_month_year = sales_month_year.sort_values(["Year", "Month Name"])

fig2 = px.bar(
    sales_month_year,
    x="Month Name",
    y="Total_Sales",
    color="Year",
    barmode="group",
    title="Monthly Sales Trend by Year"
)
fig2.show()


In [28]:
# ===============================
# 10. Sales Efficiency Metrics
# ===============================
sales_efficiency = (
    df.groupby("Product", as_index=False)
    .agg(
        Avg_Sales_per_Unit=(" Sales", lambda x: x.sum() / df.loc[x.index, "Units Sold"].sum()),
        Avg_Discount=("Discount_Rate", "mean"),
        Total_Sales=(" Sales", "sum")
    )
)

print(sales_efficiency.sort_values("Total_Sales", ascending=False))

     Product  Avg_Sales_per_Unit  Avg_Discount   Total_Sales
3      Paseo               97.60          0.07 33,011,143.95
4        VTT              121.53          0.07 20,511,921.02
5       Velo              112.36          0.07 18,250,059.46
0   Amarilla              114.27          0.08 17,747,116.06
2    Montana               99.81          0.07 15,390,801.88
1  Carretera               94.08          0.07 13,815,307.88


In [29]:
features = [
    "Units Sold",
    " Sales",
    "COGS",
    "Profit",
    "Discount_Rate",
    "Profit_per_Unit",
    "Manufacturing_Efficiency"
]

X = df[features].copy()
X = X.fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [30]:
silhouette_scores = []

for k in range(2, 7):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append((k, score))

silhouette_df = pd.DataFrame(
    silhouette_scores, columns=["Clusters", "Silhouette Score"]
)
silhouette_df


C:\Users\precious\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.

C:\Users\precious\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.

C:\Users\precious\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.

C:\Users\precious\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less 

,Clusters,Silhouette Score
0,2,0.48
1,3,0.41
2,4,0.25
3,5,0.26
4,6,0.27


In [31]:
kmeans = KMeans(n_clusters=4, random_state=42)
df["Cluster"] = kmeans.fit_predict(X_scaled)


C:\Users\precious\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [32]:
fig = px.scatter(
    df,
    x=" Sales",
    y="Profit",
    color="Cluster",
    size="Units Sold",
    hover_data=["Product", "Country"],
    title="Customer & Product Profitability Clusters"
)
fig.show()


In [33]:
cluster_summary = (
    df.groupby("Cluster")
    .agg(
        Avg_Sales=(" Sales", "mean"),
        Avg_Profit=("Profit", "mean"),
        Avg_Discount=("Discount_Rate", "mean"),
        Avg_Efficiency=("Manufacturing_Efficiency", "mean"),
        Count=("Cluster", "count")
    )
)
cluster_summary


,Avg_Sales,Avg_Profit,Avg_Discount,Avg_Efficiency,Count
Cluster,,,,,
0,"86,149.87","8,410.05",0.07,4.22,227
1,"673,422.32","110,090.90",0.07,17.22,88
2,"341,727.42","48,456.26",0.07,67.64,65
3,"55,302.76","6,709.26",0.08,3.66,320


REGRESSION MODEL – Profit Prediction
Objective

Predict profit to:

support pricing

improve forecasting

guide discount decisions

CLASSIFICATION MODEL – Loss vs Profitable Sales
Business Question

“Can we detect loss-making transactions early?”

In [34]:
df["Is_Loss"] = (df["Profit"] < 0).astype(int)
y_class = df["Is_Loss"]


In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_class, test_size=0.25, random_state=42
)

clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred_class = clf.predict(X_test)


In [36]:
features = [
    "Units Sold",
    "Manufacturing Price",
    "Sale Price",
    "Discount_Rate",
    "Manufacturing_Efficiency"
]

X = df[features]
y = df["Profit"]

X = X.fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42
)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

r2, mae


(0.509502347679669, 21096.44672067509)

In [38]:
import plotly.express as px

residuals = y_test - y_pred

fig = px.scatter(
    x=y_pred,
    y=residuals,
    title="Residual Analysis – Profit Prediction Model",
    labels={"x": "Predicted Profit", "y": "Residuals"}
)
fig.show()


In [39]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

param_grid = {"alpha": [0.01, 0.1, 1, 10, 100]}

ridge = GridSearchCV(
    Ridge(),
    param_grid,
    scoring="r2",
    cv=5
)

ridge.fit(X_train, y_train)

ridge.best_params_, ridge.best_score_


({'alpha': 10}, np.float64(0.46446941412982845))

In [40]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    "n_estimators": [200, 400],
    "max_depth": [8, 12, 16],
    "min_samples_split": [5, 10],
}

rf = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid,
    scoring="r2",
    cv=5,
    n_jobs=-1
)

rf.fit(X_train, y_train)

rf.best_params_, rf.best_score_


({'max_depth': 12, 'min_samples_split': 5, 'n_estimators': 400},
 np.float64(0.9353863446649535))

In [41]:
df["Price_Margin"] = df["Sale Price"] - df["Manufacturing Price"]
df["Revenue_per_Unit"] = df[" Sales"] / df["Units Sold"]

new_features = [
    "Units Sold",
    "Discount_Rate",
    "Manufacturing_Efficiency",
    "Price_Margin",
    "Revenue_per_Unit"
]

X = df[new_features].fillna(0)
X_scaled = scaler.fit_transform(X)


In [42]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(
    rf.best_estimator_,
    X_scaled,
    y,
    cv=5,
    scoring="r2"
)

cv_scores.mean(), cv_scores.std()


(np.float64(0.8293036247958092), np.float64(0.19350205429559458))

In [43]:
import pandas as pd

feature_importance = pd.DataFrame({
    "Feature": new_features,
    "Importance": rf.best_estimator_.feature_importances_
}).sort_values("Importance", ascending=False)

feature_importance


,Feature,Importance
2,Manufacturing_Efficiency,0.64
0,Units Sold,0.30
3,Price_Margin,0.06
4,Revenue_per_Unit,0.00
1,Discount_Rate,0.00
